# Code for PCA

In [ ]:
import pandas as pd
crispr = pd.read_csv('../../../24Q4/CRISPRGeneEffect.csv',index_col=0)
crispr.columns = [c.split()[0] for c in crispr.columns]
crispr.dropna(axis=1,inplace=True)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA


def analyze_pca_2std(data, n_permutations=10):
    X = data.values

    # --- Real PCA (ALL components) ---
    print("Running real PCA...")
    pca = PCA(n_components=None, svd_solver="full")
    pca.fit(X)

    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_.T  # genes × PCs

    # --- Null distribution ---
    print(f"Running {n_permutations} permutations...")
    random_evals = []

    for _ in range(n_permutations):
        X_shuff = X.copy()
        for j in range(X.shape[1]):
            np.random.shuffle(X_shuff[:, j])

        pca_null = PCA(n_components=None, svd_solver="full")
        pca_null.fit(X_shuff)
        random_evals.append(pca_null.explained_variance_)

    random_evals = np.vstack(random_evals)

    mean_null = random_evals.mean()
    std_null = random_evals.std()

    for i in range(0,10):
        threshold = mean_null + i * std_null
        significant_indices = np.where(eigenvalues > threshold)[0]
        print(f"Threshold (Mean + {i}SD): {threshold:.4f}")
        print(f"Significant components: {len(significant_indices)}")

    return eigenvalues, eigenvectors, significant_indices


def eigenvector_loadings_df(data, eigenvectors, eigenvalues):
    gene_names = data.columns
    colnames = [f"PC{i+1}" for i, val in enumerate(eigenvalues)]
    return pd.DataFrame(eigenvectors, index=gene_names, columns=colnames)


# --- Usage for all genes ---
eigenvalues, eigenvectors, significant_indices = analyze_pca_2std(
    crispr,
    n_permutations=10
)

df_load = eigenvector_loadings_df(crispr, eigenvectors, eigenvalues)

In [ ]:
df_load.sort_values(by='PC1', ascending=False)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4,4))
#plt.scatter(crispr['AIRIM'],crispr['YPEL5'],s=3) #PC1
plt.scatter(crispr['MDM2'],crispr['TP53'],s=3) #PC5



# Gprofiler

In [ ]:
result = {}

for col in df_load.columns:
    series = df_load[col]

    max_val = series.max()
    min_val = series.min()

    big_idx = series[series > 0.5 * max_val].index.tolist()
    small_idx = series[series < 0.5 * min_val].index.tolist()

    result[f"{col}_big"] = big_idx
    result[f"{col}_small"] = small_idx

In [ ]:
from gprofiler import GProfiler
#for a in result['EV1_small']:
xs = {}
gp = GProfiler(return_dataframe=True)

for r in [1,2,3,4,5,10,20,50,62,91,100,156,305,1000]:
    print(r)
    x = gp.profile(organism='hsapiens',
            query=result[f'PC{r}_big'])
    xs[f'PC{r}_big'] = x
    x = gp.profile(organism='hsapiens',
            query=result[f'PC{r}_small'])
    xs[f'PC{r}_small'] = x

In [ ]:
gos = []
for key in xs:
    x = xs[key]
    x_go = x[x['source'].str.startswith('GO')]
    x_go = x_go[x_go['term_size'] < 2000] # exclude very broad terms (e.g. cytoplasm)
    x_go = x_go[x_go['precision'] > 0.2] # at least 20% of genes in query

    names = x_go['name'].tolist()
    if len(names) > 0:
        print(f"{key}: {names[0]}")
        gos.append(names[0])
    else:
        print(f"{key}: None")
        gos.append(None)


# GLS gene only

In [ ]:
gls_genes = open('gls_genes.txt').read().splitlines()
gls_crispr = crispr[gls_genes]

In [ ]:
#PCA for GLS genes only
eigenvalues, eigenvectors, significant_indices = analyze_pca_2std(
    gls_crispr,
    n_permutations=10
)

gls_df_load = eigenvector_loadings_df(gls_crispr, eigenvectors, eigenvalues)

In [ ]:
gls_df_load.sort_values(by='PC1', ascending=False)
results = {}
for c in df_load.columns:
    topg = df_load[c].sort_values(ascending=False).index.tolist()[:5]
    botg = df_load[c].sort_values(ascending=False).index.tolist()[-5:]
    results[c] = topg + botg
results_df = pd.DataFrame(results)
results_df.index = ['Top1','Top2','Top3','Top4','Top5','Bot5','Bot4','Bot3','Bot2','Bot1']

In [ ]:
results_df